In [ ]:
from rfpipe import pipeline
# Updated for rfpipe version 1.3.1
from rfpipe import candidates
import rfpipe
import numpy as np 
import pylab as plt
import matplotlib
import logging
import glob
logger = logging.getLogger('rfpipe')
%matplotlib inline
from refine_utils import cd_refine, max_timewindow

In [ ]:
params = {
        'axes.labelsize' : 14,
        'font.size' : 9,
        'legend.fontsize': 12,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12,
        'text.usetex': False,
        'figure.figsize': [20, 15]
        }
matplotlib.rcParams.update(params)

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

# Set up data paths

In [ ]:
datadir = '/lustre/aoc/projects/fasttransients/staging/'
sdmname = 'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566269152540'
#'realfast_19A-397.sb37006961.eb37092350.58715.036993946764_1566270772899'
workdir = '/lustre/aoc/observers/nm-10222/rfgpu/Refinement/'
gainpath = '/home/mchammer/evladata/telcal/'

In [ ]:
datasetId = '{0}'.format('_'.join(sdmname.split('_')[1:-1]))
# # set the paths to the gainfile
gainname = datasetId + '.GN'
logging.info('Searching for the gainfile {0} in {1}'.format(gainname, gainpath))
for path, dirs, files in os.walk(gainpath):
    for f in filter(lambda x: gainname in x, files):
        gainfile = os.path.join(path, gainname)
        break

In [ ]:
gainfile = '/home/mchammer/evladata/telcal/2019/08/19A-397.sb37006961.eb37092350.58715.036993946764.GN'

# Inputs 

In [ ]:
preffile = datadir+'realfast.yml' # Not being used currently
search_sigma = 7
classify = True
sdmscan = 1
dm_steps = 100
dm_frac = 0.2

In [ ]:
# prefs={'gainfile': gainfile, 'saveplots': False, 'savenoise': False, 'savesols': False,
#        'savecandcollection': False, 'savecanddata': False,
#        'applyonlineflags': True, 'fftmode': 'cuda', 'clustercands': False, 'maxdm':2000, 'dtarr':[1,2,4,8]}

# Search on a larger timewindow 

In [ ]:
try:
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, 
                            preffile = preffile, name='NRAOdefaultS')
except AssertionError:
    prefs['maxdm'] = 500
    st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, 
                            preffile = preffile, name='NRAOdefaultS')

st.prefs.sigma_image1 = search_sigma
st.prefs.workdir = workdir
st.prefs.gainfile = gainfile
st.prefs.saveplots = False

# Uncomment the following line to run search without RFI flagging. 
# st.prefs.flaglist = []

tw = max_timewindow(st)
if tw > st.prefs.timewindow:
    try:
        logging.info('Setting timewindow to {0}'.format(tw))
        st.prefs.timewindow = tw
    except AssertionError:
        pass

In [ ]:
cc = rfpipe.pipeline.pipeline_scan(st)

In [ ]:
try:
    assert len(cc.locs) 
except AssertionError as err:
    logging.exception("No candidates found in the search.")
    raise err

In [ ]:
# Cluster and visualize Candidates
cc.prefs.clustercands = True
cc_clustered, clusterer = rfpipe.candidates.cluster_candidates(cc, returnclusterer=True)
rfpipe.candidates.visualize_clustering(cc_clustered, clusterer)

## Reproduce candidate and generate refinement plots

In [ ]:
clusters = cc_clustered.array['cluster'].astype(int)
cl_rank, cl_count = candidates.calc_cluster_rank(cc_clustered)
calcinds = np.unique(np.where(cl_rank == 1)[0]).tolist()
logging.info("Reproducing cands at {0} cluster peaks of SNR: {1}".format(len(calcinds),
                                                                         cc_clustered.snrtot[calcinds]))

In [ ]:
for ind in calcinds:
    candloc = cc_clustered.locs[ind]
    cd = rfpipe.reproduce.pipeline_canddata(st, candloc)
    logging.info(f'Processing candidate at candloc {cd.loc}')
    if cd.data.any():
        if classify:
            payload = candidates.cd_to_fetch(cd, classify=True, save_png=False, show=False, mode = 'GPU')
            logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
        logging.info('Generating Refinement plots')
        cd_refine(cd)
    else:
        logging.warning('Canddata is empty. Skipping Candidate')    

# Search with finer DM grid

In [ ]:
# DM of the candidate to be refined. 
dm = 100

In [ ]:
#set the "refined" preferences
curr_prefs = st.prefs
curr_prefs.dmarr = list(np.linspace(dm-dm_frac*dm, dm+dm_frac*dm, dm_steps))
st = rfpipe.state.State(sdmfile=os.path.join(datadir, sdmname), sdmscan=sdmscan, inprefs=curr_prefs)
cc = rfpipe.pipeline.pipeline_scan(st)

In [ ]:
try:
    assert len(cc.locs) 
except AssertionError as err:
    logging.exception("No candidates found in the search.")
    raise err

In [ ]:
# Visualise clustering
cc.prefs.clustercands = True
cc_clustered, clusterer = rfpipe.candidates.cluster_candidates(cc, returnclusterer=True)
rfpipe.candidates.visualize_clustering(cc_clustered, clusterer)

In [ ]:
clusters = cc_clustered.array['cluster'].astype(int)
cl_rank, cl_count = candidates.calc_cluster_rank(cc_clustered)
calcinds = np.unique(np.where(cl_rank == 1)[0]).tolist()
logging.info("Reproducing cands at {0} cluster peaks of SNR: {1}".format(len(calcinds),
                                                                         cc_clustered.snrtot[calcinds]))

# Reproduce and Generate refinement plots of fine-DM searched candidates

In [ ]:
for ind in calcinds:
    candloc = cc_clustered.locs[ind]
    cd = rfpipe.reproduce.pipeline_canddata(st, candloc)
    logging.info(f'Processing candidate at candloc {cd.loc}')
    if cd.data.any():
        if classify:
            payload = candidates.cd_to_fetch(cd, classify=True, save_png=False, show=False, mode = 'GPU')
            logging.info('FETCH FRB Probability of the candidate {0} is {1}'.format(cd.candid, payload))
        logging.info('Generating Refinement plots')
        cd_refine(cd)
    else:
        logging.warning('Canddata is empty. Skipping Candidate')    